In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import os
import glob
import requests
import pandas as pd
from PIL import Image
from array import array
from tqdm.auto import tqdm

from data.api_keys import *

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
computervision_client


In [2]:
encompass_images = glob.glob("./data/encompass/*/*.jpg")
camp_images = glob.glob("./data/camp/*/*.jpg")
print(f"encompass: {len(encompass_images)}\tcamp: {len(camp_images)}")

encompass: 21588	camp: 10273


In [3]:
# test_image = encompass_images[0]
# test_image

In [4]:
# image_data = open(test_image, "rb").read()
# analyze_url = endpoint + "vision/v3.2/analyze?visualFeatures=Categories,Objects,Tags&model-version=latest"
# headers = {'Ocp-Apim-Subscription-Key': subscription_key,
#            'Content-Type': 'application/octet-stream'}
# response = requests.post(
#     analyze_url, headers=headers, data=image_data)
# response.raise_for_status()

# analysis = response.json()
# print(analysis)

In [7]:
class Analyzer:
    def __init__(self, endpoint, subscription_key):
        self.analyze_url = endpoint + "vision/v3.2/analyze?visualFeatures=Categories,Objects,Tags&model-version=latest"
        self.headers = {'Ocp-Apim-Subscription-Key': subscription_key,
                       'Content-Type': 'application/octet-stream'}
        
        self.col_names = ["dataset", "ID", "number", "fname", "categories", "tags", "objects"]
        self.df = pd.DataFrame([], columns=self.col_names)
        
    def run(self, image_path, index):
        meta_ = image_path.split("/")
        meta = [meta_[2], meta_[3], int(index), meta_[-1]]
        
        image = open(image_path, "rb").read()
        response = requests.post(
            self.analyze_url, headers=self.headers, data=image)
        response.raise_for_status()

        results = response.json()
        self.append_results(meta, results)
    
    
    def append_results(self, meta, results):
        dataset, ID, number, fname = meta
        categories = results['categories']
        tags = results['tags']
        objects = results['objects']
        
        self.df = self.df.append({self.col_names[0]: dataset, 
                                  self.col_names[1]: ID, 
                                  self.col_names[2]: number, 
                                  self.col_names[3]: fname, 
                                  self.col_names[4]: categories, 
                                  self.col_names[5]: tags, 
                                  self.col_names[6]: objects}, ignore_index=True)
        

In [ ]:
analyzer = Analyzer(endpoint, subscription_key)

for index, image in enumerate(tqdm(encompass_images)):
    analyzer.run(image, index)

for index, image in enumerate(tqdm(camp_images)):
    analyzer.run(image, index)
    
analyzer.df.to_csv("data/image_information_azure.csv", index=False)

  0%|          | 0/21588 [00:00<?, ?it/s]